In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import joblib


In [5]:
df = pd.read_csv(r"C:\Users\Laptop\Desktop\walmart_data_anlytic project\data\Walmart_cleaned_data.csv")  

df.head()

,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin,total
0,1,WALM003,San Antonio,Health and beauty,74.69,7.0,05/01/19,13:08:00,Ewallet,9.1,0.48,522.83
1,2,WALM048,Harlingen,Electronic accessories,15.28,5.0,08/03/19,10:29:00,Cash,9.6,0.48,76.40
2,3,WALM067,Haltom City,Home and lifestyle,46.33,7.0,03/03/19,13:23:00,Credit card,7.4,0.33,324.31
3,4,WALM064,Bedford,Health and beauty,58.22,8.0,27/01/19,20:33:00,Ewallet,8.4,0.33,465.76
4,5,WALM013,Irving,Sports and travel,86.31,7.0,08/02/19,10:37:00,Ewallet,5.3,0.48,604.17


In [6]:
df['day'] = pd.to_datetime(df['date']).dt.day
df['month'] = pd.to_datetime(df['date']).dt.month
df['hour'] = pd.to_datetime(df['time']).dt.hour

C:\Users\Laptop\AppData\Local\Temp\ipykernel_8668\2270418581.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['day'] = pd.to_datetime(df['date']).dt.day
C:\Users\Laptop\AppData\Local\Temp\ipykernel_8668\2270418581.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['month'] = pd.to_datetime(df['date']).dt.month
C:\Users\Laptop\AppData\Local\Temp\ipykernel_8668\2270418581.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['hour'] = pd.to_datetime(df['time']).dt.hour


In [7]:
df[['date', 'time', 'day', 'month', 'hour']].head()

,date,time,day,month,hour
0,05/01/19,13:08:00,1,5,13
1,08/03/19,10:29:00,3,8,10
2,03/03/19,13:23:00,3,3,13
3,27/01/19,20:33:00,27,1,20
4,08/02/19,10:37:00,2,8,10


In [8]:
X = df[['Branch', 'City', 'category', 'unit_price', 'quantity', 'rating', 'day', 'month', 'hour']]
y = df['total']

In [9]:
categorical_cols = ['Branch', 'City', 'category']

In [10]:
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
], remainder='passthrough')

In [11]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42))
])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
model.fit(X_train, y_train)
print("Model training complete")

Model training complete


In [15]:
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))  # Use sqrt manually
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

RMSE: 2.54
R² Score: 1.00


In [20]:
os.makedirs("scripts", exist_ok=True)
joblib.dump(model, "scripts/xgb_sales_model.pkl")

['scripts/xgb_sales_model.pkl']